In [1]:
from transformers import LlamaForCausalLM, LlamaTokenizer
import torch

The parameters

```
padding_side="left", truncation_side="right", add_bos_token=False
```

were taken from the EasyLM llama_model.py code. It is uncertain whether it is the correct configuration, but it seems to run fine for now.


The parameter

```
torch_dtype=torch.float16
```

ensures that 16 bit float precision is used instead of 32 bit, which theoretically reduces the memory requirement to less than 16G of GPU RAM. However, in practice, 
it turned out that sometimes 16G is not sufficient perhaps due to loading the input on CUDA as well, so move up to 2 T4 or increase total GPU RAM by other means to beyond 16G. Alternatively, perhaps there's a way to clear the GPU RAM before generating to make space.

In [2]:
tokenizer = LlamaTokenizer.from_pretrained("/home/jupyter/koala_transformer", device_map="auto", padding_side="left", truncation_side="right", add_bos_token=False)
model = LlamaForCausalLM.from_pretrained("/home/jupyter/koala_transformer", torch_dtype=torch.float16, device_map="auto", cache_dir="/home/jupyter/data/transformers")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The generate function is adapted to the correct format required by https://github.com/young-geng/EasyLM/blob/main/docs/koala.md

In [5]:
def generate(prompt, max_new_tokens=1024):
    
    # Create the prompt according to the pre-trained format: see https://github.com/young-geng/EasyLM/blob/main/docs/koala.md
    
    template = "BEGINNING OF CONVERSATION: USER: %s "
    input_start = template % prompt
    input_final = template % prompt + "GPT:"
    
    inputs = tokenizer(input_final, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=max_new_tokens, 
                             do_sample=True, 
                             top_p=1.0,
                             num_beams=1,
                             top_k=50,
                             temperature=0.8)
    result = tokenizer.batch_decode(outputs, skip_special_tokens=True, spaces_between_special_tokens=False)
    
    # Since the task is Autoregressive LM, it's gonna repeat the prompt in the output. Thus, the prompt itself is removed from the output.
    
    return result[0][len(input_start):]

In [4]:
print(generate("How to make a BLT?"))

GPT: To make a BLT (bacon, lettuce, and tomato), choose your favorite bacon slices and fry them in a pan until they are crispy and cooked to your liking. Then, prepare your lettuce by rinsing it and slicing it into bite-sized pieces. In a separate pan, fry your diced tomatoes until they are softened. Assemble your BLT by spreading mayonnaise on a slice of bread and layering it with the bacon, lettuce, and tomatoes. Add a dash of salt and pepper to taste.


In [5]:
print(generate("How to make shrimp tempura?"))

GPT: To make shrimp tempura, you will need the following ingredients:

*   Shrimp, peeled and deveined
*   Cooking oil
*   Potato flour
*   Panko breadcrumbs
*   Soy sauce
*   Mirin
*   Dashi
*   Salt
*   Peanut oil
*   Shrimp tempura batter (recipe below)

<ol start="1"><li>In a large bowl, mix together the shrimp, potato flour and panko breadcrumbs. Allow the mixture to sit for 30 minutes to allow the shrimp to absorb the flavor of the ingredients.</li><li>In a separate, small bowl, mix together the soy sauce, mirin and dashi. In a separate, large bowl, mix together the soy sauce mixture, dashi mixture, and salt. Gradually add the egg mixture to the bowl while whisking.</li><li>Add the shrimp mixture to the egg mixture and stir gently to coat the shrimp evenly in the batter.</li><li>Heat the peanut oil in a large skillet over medium heat. Add the shrimp tempura to the skillet and fry the shrimp for 2-3 minutes on each side, or until they are golden brown and cooked through.</li><li>R

In [6]:
article = """Alzheimer’s disease treatment stocks are focused on Alzheimer’s disease, a degenerative brain disorder that results in declining memory and thinking skills and typically affects people in their mid-60s.According to the Alzheimer’s Association, neurons in other areas of the brain also begin to deteriorate as Alzheimer’s disease gets worse, resulting in the loss of basic human functions and overall cognitive impairment.This condition affects more than 6.7 million people in the US alone; it’s also the most common form of dementia and is the sixth leading cause of death in America. Treatments are available to alleviate Alzheimer’s disease symptoms, but because they do not affect the underlying causes of this neurodegenerative disease, they’re only a bandaid solution.Therapies approved by the US Food and Drug Administration (FDA) include: rivastigmine by Novartis (NYSE:NVS); galantamine, developed by Janssen, a division of Johnson & Johnson (NYSE:JNJ); donepezil by Pfizer (NYSE:PFE); and AbbVie's (NYSE:ABBV) memantine.Since there is no cure for Alzheimer’s disease, death is often the result for patients as the ailment devastates the brain. And unfortunately, Alzheimer’s disease is rising in prevalence — a report from Research and Markets suggests that the global Alzheimer’s disease treatment market will be worth a significant US$9.64 billion by 2028 as more patients need treatment, and as more investments are made in biomarkers for diagnosis and drug development.Here the Investing News Network takes a look at a few of the biggest Alzheimer’s disease treatment stocks on the NASDAQ. Companies are listed in order of market cap from largest to smallest, and all data was current as of April 19, 2023.. Biogen (NASDAQ:BIIB) Company Profile.Market cap: US$42.27 billion; current share price: US$292.80.Massachusetts-based Biogen, a pioneer in the field of neuroscience, is developing, manufacturing and marketing therapies aimed at treating serious neurological, neurodegenerative, autoimmune and rare diseases.The global biotechnology firm’s research areas include Alzheimer's disease and dementia. However, Biogen’s launch of its FDA-approved Alzheimer’s disease drug Aduhelm faced a lot of pushback in 2022, both from the market and from Congress, over what was viewed as a hasty fast-track approval process and exorbitant costs to patients.In early 2023, Biogen is looking to give it another go, this time with LEQEMBI (lecanemab-irmb) for the treatment of Alzheimer's disease, which the FDA has approved under its accelerated approval pathway. The drug was jointly developed by Biogen and Tokyo-based pharmaceutical company Eisai (OTC Pink:ESALF,TSE:4523). Intra-Cellular Therapies (NASDAQ:ITCI).Company Profile.Market cap: US$5.96 billion; current share price: US$62.59.As its name suggests, biopharma company Intra-Cellular Therapies is taking an intracellular approach to developing therapies for patients with complex psychiatric and neurologic diseases, including Parkinson’s disease and Alzheimer’s disease.Intra-Cellular Therapies is currently evaluating ITI-1284-ODT-SL, a deuterated form of lumateperone that comes in an oral disintegrating tablet; ITI-1284-ODT-SL is in Phase 1 studies, including drug-drug interaction studies. The company expects to commence Phase 2 clinical trials in agitation in patients with probable Alzheimer’s disease in 2023. 3. Axsome Therapeutics (NASDAQ:AXSM).Company Profile.Market cap: US$3.16 billion; current share price: US$72.64.New York-based Axsome Therapeutics is developing and commercializing a portfolio of novel therapies for disorders of the central nervous system with limited treatment options. Products currently on the market from the biopharmaceutical company include Auvelity, a rapid-acting oral antidepressant for the treatment of major depressive disorder; and Sunosi, a once-daily prescription medicine to treat excessive daytime sleepiness due to obstructive sleep apnea or narcolepsy.Axsome’s product pipeline includes AXS-05, a NMDA receptor antagonist with multimodal activity. It has breakthrough therapy designation from the FDA and is currently in Phase 3 clinical studies for the treatment of Alzheimer’s disease agitation. """

response = generate("You are an equity researcher that will advise personal investors on their portfolio. Summarize the following article. Focus on the information that can impact the investment portfolio or the investment decision.\n" + article)
print(response)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 article = """Alzheimer’s disease treatment stocks are focused on Alzheimer’s disease, a      │
│   2                                                                                              │
│ ❱ 3 response = generate("You are an equity researcher that will advise personal investors on     │
│   4 print(response)                                                                              │
│   5                                                                                              │
│                                                                                                  │
│ in generate:15                                                                                   │
│                                                                                                  │
│   12 │   │   │   │   │   │   │    top_p=1.0,                                                     │
│   13 │   │   │   │   │   │   │    num_beams=1,                                                   │
│   14 │   │   │   │   │   │   │    top_k=50,                                                      │
│ ❱ 15 │   │   │   │   │   │   │    temperature=0.8)                                               │
│   16 │   result = tokenizer.batch_decode(outputs, skip_special_tokens=True, spaces_between_sp    │
│   17                                                                                             │
│   18                                                                                             │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/torch/autograd/grad_mode.py:27 in decorate_context        │
│                                                                                                  │
│    24 │   │   @functools.wraps(func)                                                             │
│    25 │   │   def decorate_context(*args, **kwargs):                                             │
│    26 │   │   │   with self.clone():                                                             │
│ ❱  27 │   │   │   │   return func(*args, **kwargs)                                               │
│    28 │   │   return cast(F, decorate_context)                                                   │
│    29 │                                                                                          │
│    30 │   def _wrap_generator(self, func):                                                       │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/transformers/generation/utils.py:1496 in generate         │
│                                                                                                  │
│   1493 │   │   │   │   return_dict_in_generate=generation_config.return_dict_in_generate,        │
│   1494 │   │   │   │   synced_gpus=synced_gpus,                                                  │
│   1495 │   │   │   │   streamer=streamer,                                                        │
│ ❱ 1496 │   │   │   │   **model_kwargs,                                                           │
│   1497 │   │   │   )                                                                             │
│   1498 │   │                                                                                     │
│   1499 │   │   elif is_beam_gen_mode:                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.7/site-packages/transformers/generat